# SYMFLUENCE Tutorial 02c — Basin-Scale Workflow (Bow River at Banff, Elevation-Based Distributed)

## Introduction

This tutorial demonstrates the most spatially-detailed modeling approach: elevation-based HRU discretization. Building on Tutorials 02a (lumped) and 02b (semi-distributed), we now subdivide each GRU into elevation bands that capture altitudinal controls on mountain hydrology.

Elevation-based discretization is critical in mountain watersheds where temperature and precipitation vary systematically with altitude. By stratifying each sub-basin into elevation bands, we better represent snowpack dynamics, seasonal timing differences, and orographic effects.

The key configuration parameter is `ELEVATION_BAND_SIZE`, which controls the vertical resolution (e.g., 100m bands). Smaller bands increase spatial detail but add computational cost. This approach maintains the validated GRU structure from Tutorial 02b while adding vertical stratification.

For the **Bow River at Banff** (elevation range: 1,384–3,400 m), elevation bands capture the transition from low-elevation rain-dominated zones to high-elevation snow-dominated headwaters, improving simulation of spring freshet timing and runoff generation.


# Step 1 — Configuration and data reuse

We configure elevation-based discretization and reuse data from Tutorial 02b where possible.

In [ ]:
# Step 1 — Elevation-based configuration with data reuse


from pathlib import Path
import yaml
import shutil


from symfluence import SYMFLUENCE
from symfluence.resources import get_config_template

SYMFLUENCE_CODE_DIR = Path.cwd().resolve()
# Load template to get the correct SYMFLUENCE_DATA_DIR
config_template = get_config_template()
with open(config_template, 'r') as f:
    template_config = yaml.safe_load(f)

# Use SYMFLUENCE_DATA_DIR from template
SYMFLUENCE_DATA_DIR = Path(template_config.get('SYMFLUENCE_DATA_DIR', 
                                                 str(SYMFLUENCE_CODE_DIR.parent / 'data' / 'SYMFLUENCE_data'))).resolve()

print(f"Using SYMFLUENCE_DATA_DIR: {SYMFLUENCE_DATA_DIR}")

# Load template again for config creation
with open(config_template, 'r') as f:
    config = yaml.safe_load(f)

# === Modify for elevation-based distributed ===
config['SYMFLUENCE_CODE_DIR'] = str(SYMFLUENCE_CODE_DIR)
config['SYMFLUENCE_DATA_DIR'] = str(SYMFLUENCE_DATA_DIR)
config['DOMAIN_NAME'] = 'Bow_at_Banff_elevation'
config['EXPERIMENT_ID'] = 'run_1'
config['POUR_POINT_COORDS'] = '51.1722/-115.5717'

# Elevation-based discretization
config['DOMAIN_DEFINITION_METHOD'] = 'delineate'
config['STREAM_THRESHOLD'] = 5000  # Same as 02b
config['DOMAIN_DISCRETIZATION'] = 'elevation'  # Key change
config['ELEVATION_BAND_SIZE'] = 400  # 400m elevation bands

config['HYDROLOGICAL_MODEL'] = 'SUMMA'
config['ROUTING_MODEL'] = 'mizuRoute'
config['MIZU_FROM_MODEL'] = 'SUMMA'

# mizuRoute routing configuration
config['SETTINGS_MIZU_ROUTING_VAR'] = 'averageRoutedRunoff'  # Input variable from SUMMA
config['SETTINGS_MIZU_ROUTING_UNITS'] = 'm/s'
config['SETTINGS_MIZU_ROUTING_DT'] = '3600'
# Routing scheme: 0=accumRunoff, 1=IRF-UH (recommended), 2=IRF-KW, 3=KW-IRF, 4=MC-IRF
# IRF (Impulse Response Function) applies proper river routing delays
config['SETTINGS_MIZU_OUTPUT_VARS'] = 1  # Use IRF routing for proper streamflow evaluation

# Temporal extent
config['EXPERIMENT_TIME_START'] = '2004-01-01 01:00'
config['EXPERIMENT_TIME_END'] = '2007-12-31 23:00'
config['CALIBRATION_PERIOD'] = '2005-10-01, 2006-09-30'
config['EVALUATION_PERIOD'] = '2006-10-01, 2007-12-30'
config['SPINUP_PERIOD'] = '2004-01-01, 2005-09-30'

config['STATION_ID'] = '05BB001'
config['DOWNLOAD_WSC_DATA'] = True

config['PARAMS_TO_CALIBRATE'] = 'minStomatalResistance,cond2photo_slope,vcmax25_canopyTop,jmax25_scale,summerLAI,rootingDepth,soilStressParam,z0Canopy,windReductionParam'
config['OPTIMIZATION_TARGET'] = 'streamflow'
config['ITERATIVE_OPTIMIZATION_ALGORITHM'] = 'DDS'
config['OPTIMIZATION_METRIC'] = 'KGE'
config['CALIBRATION_TIMESTEP'] = 'hourly'  

# Save configuration to current directory
config_path = Path('./config_elevation_distributed.yaml')
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False, sort_keys=False)

print(f"Configuration saved: {config_path}")

# === Data reuse from Tutorial 02b ===
semi_dist_domain = 'Bow_at_Banff_semi_distributed'
semi_dist_data_dir = SYMFLUENCE_DATA_DIR / f'domain_{semi_dist_domain}'

def copy_with_name_adaptation(src, dst, old_name, new_name):
    """Copy directory/file and adapt filenames containing old_name to new_name.
    
    Uses explicit file iteration instead of copytree for reliability with existing directories.
    """
    if not src.exists():
        return False
    
    try:
        if src.is_file():
            # Single file: copy with name adaptation
            dst.parent.mkdir(parents=True, exist_ok=True)
            new_filename = dst.name.replace(old_name, new_name) if old_name in dst.name else dst.name
            shutil.copy2(src, dst.parent / new_filename)
            return True
        else:
            # Directory: iterate and copy each file individually
            dst.mkdir(parents=True, exist_ok=True)
            copied_count = 0
            for src_file in src.rglob('*'):
                if src_file.is_file():
                    # Compute relative path and adapt names in each path component
                    rel_path = src_file.relative_to(src)
                    adapted_parts = [
                        part.replace(old_name, new_name) if old_name in part else part
                        for part in rel_path.parts
                    ]
                    dst_file = dst / Path(*adapted_parts)
                    dst_file.parent.mkdir(parents=True, exist_ok=True)
                    shutil.copy2(src_file, dst_file)
                    copied_count += 1
            return copied_count > 0
    except Exception as e:
        print(f"   Warning: Copy failed for {src.name}: {e}")
        return False

# Initialize SYMFLUENCE with visualization enabled
symfluence = SYMFLUENCE(config_path, visualize=True)
project_dir = symfluence.managers['project'].setup_project()

if semi_dist_data_dir.exists():
    print(f"\nReusing data from Tutorial 02b: {semi_dist_data_dir}")
    
    reusable_data = {
        'Elevation': semi_dist_data_dir / 'attributes' / 'elevation',
        'Land Cover': semi_dist_data_dir / 'attributes' / 'landclass',
        'Soils': semi_dist_data_dir / 'attributes' / 'soilclass',
        'Forcing': semi_dist_data_dir / 'forcing' / 'raw_data',
        'Stream Network': semi_dist_data_dir / 'shapefiles' / 'river_network',
        'GRUs': semi_dist_data_dir / 'shapefiles' / 'river_basins',
        'Streamflow': semi_dist_data_dir / 'observations' / 'streamflow'
    }
    
    for data_type, src_path in reusable_data.items():
        if src_path.exists():
            rel_path = src_path.relative_to(semi_dist_data_dir)
            dst_path = project_dir / rel_path
            success = copy_with_name_adaptation(src_path, dst_path, semi_dist_domain, config['DOMAIN_NAME'])
            if success:
                print(f"   {data_type}: Copied")
        else:
            print(f"   {data_type}: Not found")
else:
    print(f"\nNo data from Tutorial 02b found at: {semi_dist_data_dir}")
    print(f"   Will acquire fresh data.")

# Create pour point
pour_point_path = symfluence.managers['project'].create_pour_point()
print(f"\nProject structure created at: {project_dir}")

## Step 2 — Elevation-based discretization

Subdivide each GRU from Tutorial 02b into elevation bands for vertical stratification.

### Step 2a — Attribute check

Verify DEM and GRU availability from data reuse.

In [ ]:
# Step 2a — DEM and GRU availability check
dem_path = project_dir / 'attributes' / 'elevation' / 'dem'
gru_path = project_dir / 'shapefiles' / 'river_basins'

if not dem_path.exists() or not gru_path.exists():
    print("   Required data not found, acquiring...")
    # If using MAF supported HPC, uncomment the lines below
    # symfluence.managers['data'].acquire_attributes()
    # symfluence.managers['domain'].define_domain()
    print("✅ Geospatial data acquired")
else:
    print("✅ DEM and GRU data available from previous workflow")

### Step 2b — Elevation band creation

Create HRUs by intersecting GRUs with elevation bands.

In [ ]:
# Step 2b — Elevation-based HRU discretization
hru_path = symfluence.managers['domain'].discretize_domain()
print("✅ Elevation-based HRU discretization complete")

### Step 2c — Elevation structure visualization

Visualize the elevation-stratified HRU structure.

In [ ]:
# Step 2c — Elevation band visualization (using native SYMFLUENCE plotting)

from IPython.display import Image, display

# Generate native discretized domain visualization (elevation bands)
plot_path = symfluence.managers['domain'].visualize_discretized_domain()

if plot_path:
    print(f"Domain plot saved to: {plot_path}")
    display(Image(filename=str(plot_path)))
else:
    print("Domain visualization not generated. Check logs for errors.")

## Step 3 — Data preprocessing

Process forcing and observation data for elevation-stratified HRUs.

In [ ]:
# Step 3a — Streamflow observations
# If using MAF supported HPC, uncomment the line below
# symfluence.managers['data'].process_observed_data()
print("✅ Streamflow data processing complete")

In [ ]:
# Step 3b — Forcing data
# If using MAF supported HPC, uncomment the line below
# symfluence.managers['data'].acquire_forcings()
print("✅ Forcing acquisition complete")

In [ ]:
# Step 3c — Model-agnostic preprocessing
symfluence.managers['data'].run_model_agnostic_preprocessing()
print("✅ Model-agnostic preprocessing complete")

## Step 4 — Model execution

Configure and run SUMMA-mizuRoute with elevation-stratified HRUs.

In [ ]:
# Step 4a — Model configuration
symfluence.managers['model'].preprocess_models()
print("✅ Elevation-based model configuration complete")

In [ ]:
# Step 4b — Model execution
symfluence.managers['model'].run_models()
print("✅ Elevation-based distributed simulation complete")

## Step 5 — Evaluation

Compare elevation-based results against observations and previous approaches.

In [ ]:
# Step 5 — Elevation-based evaluation (using Camille's model comparison plots)

from IPython.display import Image, display

# Generate model comparison overview (auto-detects mizuRoute outputs)
plot_path = symfluence.managers['reporting'].generate_model_comparison_overview(
    experiment_id=config['EXPERIMENT_ID'],
    context='run_model'
)

if plot_path:
    print(f"Model comparison overview: {plot_path}")
    display(Image(filename=str(plot_path)))
else:
    print("No model outputs found for comparison. Check simulation outputs.")

print("\nElevation-based evaluation complete")

# Step 5b — Run calibration 



In [ ]:
results_file = symfluence.managers['optimization'].calibrate_model()  
print("Calibration results file:", results_file)

In [ ]:
# Step 5c — Post-calibration visualization 
from IPython.display import Image, display

# Generate post-calibration visualizations
plot_paths = symfluence.managers['reporting'].visualize_calibration_results(
    experiment_id=config['EXPERIMENT_ID']
)

# Display all generated plots
for plot_name, plot_path in plot_paths.items():
    print(f"\n{plot_name}:")
    display(Image(filename=str(plot_path)))

print("\nPost-calibration visualization complete")